In [ ]:
# Data Manipulation
import numpy as np
import pandas as pd
# Visualisation
import matplotlib.pyplot as plt
# Dataset exploring
import os
# Dataset generation
from keras.preprocessing import image_dataset_from_directory
from keras.preprocessing.image import ImageDataGenerator
# Transfert learning
from keras.applications import VGG16
# Optimizer
from keras.optimizers import Adam
# Keras layers
from keras.layers import Input, Dense, Dropout, Flatten, AveragePooling2D
# Keras model
from keras.models import Model
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
import keras


#requirements before frontend code
#load the paths

TRAIN_PATH = r'C:\Users\yogi\Downloads\archive\dataset\train'
TEST_PATH = r'C:\Users\yogi\Downloads\archive\dataset\test'

datagen = ImageDataGenerator(
    rotation_range = 30, 
    zoom_range = .3, 
    horizontal_flip = True, 
    vertical_flip = True, 
    validation_split = .3
)

train_ds = datagen.flow_from_directory(
    directory = TRAIN_PATH,
    target_size = (256, 256),
    color_mode = 'rgb',
    class_mode = 'categorical',
    subset = 'training'
)

validation_ds = datagen.flow_from_directory(
    directory = TRAIN_PATH,
    target_size = (256, 256),
    color_mode = 'rgb',
    class_mode = 'categorical',
    subset = 'validation'
)

#load the class label
class_labels = list(train_ds.class_indices.keys())

# Load the model without weights
model = load_model("Rotten_fruit_detection_model.h5", compile=False)

# Define the new input shape
new_input_shape = (256, 256, 3)

# Create a new VGG16 model with the modified input shape
new_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=new_input_shape)
new_vgg16.trainable = False  # Set VGG16 to be non-trainable

# Build a new model by combining the new VGG16 and the remaining part of the original model
new_model = keras.models.Sequential([
    new_vgg16,
    model.get_layer('AvgPool2D'),  
    model.get_layer('Flatten'),    
    model.get_layer('Dense1'),   
    model.get_layer('dropout'),    
    model.get_layer('Dense2'),     
    model.get_layer('dropout_1'),  
    model.get_layer('Dense3')      
])

# Compile the new model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Declare tk_img as a global variable
tk_img = None

def upload_image():
    global tk_img
    file_path = filedialog.askopenfilename(title="Select Image", filetypes=[("Image files", "*.png;*.jpg;*.jpeg")])
    if file_path:
        img = Image.open(file_path)
        img.thumbnail((256, 256))
        tk_img = ImageTk.PhotoImage(img)
        canvas.itemconfig(image_item, image=tk_img)  # Update the canvas image
        label_upload.config(text="Image Uploaded")

        # Save image path for later use
        analyze_image.image_path = file_path

def analyze_image():
    if hasattr(analyze_image, 'image_path'):
        img = Image.open(analyze_image.image_path)
        img = img.resize((256, 256))  # Resize the image to the model's expected input shape
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)

        predictions = new_model.predict(img_array)
        predicted_class_index = np.argmax(predictions)
        predicted_class_label = class_labels[predicted_class_index]

        analyze_image.predicted_class_label = predicted_class_label
        result_label.config(text=f'Prediction: {predicted_class_label}')
    else:
        result_label.config(text="Please upload an image first")

# Create the main window
root = tk.Tk()
root.title("Farmer Buddy App")

# Set the background color of the canvas
canvas_bg_color = "#f0f0f0"
canvas = tk.Canvas(root, width=400, height=400, bg=canvas_bg_color)
canvas.pack()

# Create bold label with the app name
app_name_label = tk.Label(root, text="Farmer Buddy App", font=("Helvetica", 20, "bold"))
app_name_label.pack(pady=10)

# Create labels, buttons, and canvas
label_upload = tk.Label(root, text="Upload Image:", font=("Helvetica", 16))
label_upload.pack(pady=10)

btn_upload = tk.Button(root, text="Browse", command=upload_image, font=("Helvetica", 14))
btn_upload.pack(pady=10)

btn_analyze = tk.Button(root, text="Analyze", command=analyze_image, font=("Helvetica", 14))
btn_analyze.pack(pady=10)

result_label = tk.Label(root, text="", font=("Helvetica", 16))
result_label.pack(pady=10)

# Initialize variables
analyze_image.image_path = ""
analyze_image.predicted_class_label = ""

# Create an initial empty image on the canvas
img = Image.new("RGB", (256, 256), "white")
default_img = ImageTk.PhotoImage(img)
image_item = canvas.create_image(200, 200, anchor="center", image=default_img)

# Start the Tkinter event loop
root.mainloop()


Found 7634 images belonging to 6 classes.
Found 3267 images belonging to 6 classes.
1/1 [==============================] - 0s 236ms/step
